In [1]:
#My VS Code has issues connecting to the venv.
# I find that first connecting to one of my conda environments
# and then switching to the venv works.
# I run this cell on conda environment first then switch to venv
# and run it there and seems to be all good.
print("Hello")

Hello


In [2]:
# This cell allows hot reloading of modules.
# It is useful for development, so you don't have to restart the kernel.
# Some changes such as adding new files or changing the structure of the code
# may not be picked up and will require a kernel restart.
%load_ext autoreload
%autoreload 2

# Testing LINZ with GeoPandas  
This notebook is for testing a python environment that has GeoPandas installed, using LINZ as the testing portal.  

This notebook expects GeoPandas and not the arcgis module. This will force the default data types to use GeoDataFrames.  

In [15]:
import os
import json
import time
import geopandas as gpd
from dotenv import load_dotenv, find_dotenv
import logging

# Set up logging include logger name
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

from kapipy.gis import GISK
from kapipy.gis import has_geopandas, has_arcgis, has_arcpy

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
api_key = os.getenv('KOORDINATES_API_KEY')

output_folder = r"c:\temp\data"

#layers
nz_police_station_boundaries_2021 = "105482" #284 polygons
nz_property_titles = "50804" #24.M polygons

#tables
nz_properties_national_district_valuation_roll = "114085" #246K records

print(f'{has_arcgis=}')
print(f'{has_arcpy=}')
print(f'{has_geopandas=}')

assert(has_geopandas == True)
assert(has_arcgis == False)


has_arcgis=False
has_arcpy=False
has_geopandas=True


## Server object  
Get a reference to the LINZ server  

In [16]:
k = GISK(url="https://koordinates.com/", api_key=api_key)
print(k)

GISK: Custom at https://koordinates.com/ (API v1.x)


## Layer item  
Run some tests on a layer item  

In [17]:
itm = k.content.get(nz_property_titles)
print(itm)

2025-06-12 17:21:57,820 - httpx - INFO - HTTP Request: GET https://koordinates.com/services/api/v1.x/data/?id=50804 "HTTP/1.1 200 OK"


None


In [7]:
print(f'{itm.supports_changesets=}')
print(f'{itm.data.fields=}')
print(f'{itm.data.crs=}')
print(f'{itm.data.geometry_type=}')

AttributeError: 'NoneType' object has no attribute 'supports_changesets'

In [ ]:
waikato_polygon = gpd.read_file('../examples/waikato.json')
print(waikato_polygon)

# read shapefile into a GeoDataFrame
matamata_gdf = gpd.read_file("../examples/matamata_piako.shp")
matamata_gdf.head()

In [ ]:
#Query using WFS and bbox_geometry
res = itm.query(
    result_record_count=1001,
    out_fields=['id', 'geodetic_code'],
    out_sr = 2193, 
    bbox_geometry=matamata_gdf)
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()

In [ ]:
#Query using WFS and filter_geometry
res = itm.query(
    #out_fields=['id', 'geodetic_code', itm.data.geometry_field],
    out_sr = 2193, 
    filter_geometry=matamata_gdf)
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()

In [ ]:
#Query using WFS and bbox_geometry
res = itm.query(
    out_sr = 2193, 
    bbox_geometry=matamata_gdf)
logging.info(f"Total records returned {itm.title}: {res.shape[0]}")
res.head()

In [ ]:
# Get a changeset from the layer using WFS and bbox_geometry
cs_data = itm.changeset(
    from_time="2024-01-01T00:00:00Z", 
    out_sr=2193,
    #out_fields=['id', 'geodetic_code'],
    bbox_geometry=matamata_gdf
    )
logging.info(f"Total records returned {itm.title}: {cs_data.shape[0]}")
cs_data.head()

In [ ]:
# Get a changeset from the layer using WFS and filter_geometry
cs_data = itm.changeset(
    from_time="2024-01-01T00:00:00Z", 
    out_sr=2193,
    #out_fields=['id', 'geodetic_code'],
    filter_geometry=matamata_gdf
    )
logging.info(f"Total records returned {itm.title}: {cs_data.shape[0]}")
cs_data.head()

In [ ]:
job = itm.export("geodatabase", out_sr=2193, extent=waikato_polygon)

In [ ]:
print(job.status)

In [ ]:
dl = job.download(folder=output_folder)
print(dl)

In [ ]:
print(f'{dl.folder=}')
print(f'{dl.filename=}')
print(f'{dl.file_path=}')
print(f'{dl.file_size_bytes=}')
print(f'{dl.download_url=}')
print(f'{dl.final_url=}')
print(f'{dl.job_id=}')
print(f'{dl.completed_at=}')
print(f'{dl.checksum=}')

## Table item  
Run some tests on a table item

In [ ]:
table_itm = linz.content.get(suburb_locality_table_id)
print(table_itm)

In [ ]:
print(f'{table_itm.supports_changesets=}')
print(f'{table_itm.data.fields=}')

In [ ]:
#Query using WFS to get DataFrame
data = table_itm.query(count=100)
logging.info(f"Total records returned {table_itm.title}: {data.shape[0]}")
data.head()

In [ ]:
#Query using WFS to get raw JSON
data = table_itm.query_json(count=10)
logging.info(f"Total records returned {table_itm.title}: {len(data['features'])}")
print(data)

In [ ]:
# Get a changeset from the layer using WFS
cs_data = table_itm.get_changeset(from_time="2024-01-01T00:00:00Z", count=100)
logging.info(f"Total records returned {table_itm.title}: {cs_data.shape[0]}")
cs_data.head()

In [ ]:
# Get a changeset from the layer using WFS
cs_data = table_itm.get_changeset_json(from_time="2024-01-01T00:00:00Z", count=10)
logging.info(f"Total records returned {table_itm.title}: {len(cs_data['features'])}")
print(cs_data)

In [ ]:
job = table_itm.export("geodatabase")

In [ ]:
print(job)

In [ ]:
job.download(folder=output_folder)

## Test multidownload samples  
Test downloading multiple jobs using the ContentManager download method.    

In [ ]:
itm1 = linz.content.get(geodetic_marks_layer_id)
itm2 = linz.content.get(rail_station_layer_id)
print(itm1)
print(itm2)

In [ ]:
job1 = itm1.export("geodatabase", wkid=2193, extent=matamata_gdf,)
job2 = itm2.export("geodatabase", wkid=2193, extent=matamata_gdf,)

output_folder = r"c:\temp\data\multiple"
results = linz.content.download(jobs=[job1, job2], folder=output_folder)

In [ ]:
for result in results:
    print(result.download_file_path)
